# v3afxB_test2.ipynb

Runs parallel versions of the network in `v3_nst_afxB` each with different inititial weights, reporting aggregated data about their performance.

The tests performed concern weight robustness.

In [1]:
%cd ../..
import numpy as np
import matplotlib.pyplot as plt
import time
from multiprocessing import Pool
from datetime import datetime

/home/z/projects/draculab


In [2]:
%load_ext Cython

In [3]:
%%cython
from draculab import *

In [4]:
%cd notebook/spinal/
from net_from_cfg import *
%cd ../..

/home/z/projects/draculab/notebook/spinal
/home/z/projects/draculab


In [5]:
def compare(mat1, mat2):
    """ This function "compares" its two input matrices.
    
        Args: two numpy 2D arrays mat1, mat2 of the same shape
    """
    # proportion of entries with the same sign
    n_matches = sum((np.sign(mat1) == np.sign(mat2)).flatten())
    match_prop = n_matches / mat1.size
    
    # Frobenius norm of the matrices, norm of the difference
    norm1 = np.linalg.norm(mat1)
    norm2 = np.linalg.norm(mat2)
    norm_diff = np.linalg.norm(mat1-mat2)
    #print("\n Norm 1: %f, Norm2: %f, Norm Diff: %f" % (norm1, norm2, norm_diff))
    nd_ratio = 2. * norm_diff / (norm1 + norm2)
    
    # find the angle between them
    angle = np.arccos((mat1*mat2).sum()/(norm1*norm2))
    #print("\n Angle between mats is: %f" % (angle))
    #print("\n Mean of angle with random matrices should be %f" % (np.pi/2.))
    
    return {'match_prop': match_prop, 'nd_ratio':nd_ratio, 'angle':angle }
    

In [6]:
# A function to test a single network with random initial weights

def test_net(args_dict):
    """ A function to test a single network with random initial weights. 
    
        Args: A dictionary with the following entries
            cfg : parameter dictionary to initialize the network. 
            time1 : time for first simualtion period.
            time2 : time for second simualtion period.
            avg_window : window to obtain error averages. avg_window < time1.
            err_thr : error threshold. See below.
            noisy_syns: if True, add synaptic noise
            decay: if noisy syns, do syns decay rather than drift (replaces normalization)
            
        Returns: A dictionary with the following entries
            conv_t : convergence time. Smallest value of time such that the average
                     error in (conv_t, conv_t+avg_window) is less than err_thr.
            error1: average of error in [time1-avg_window, time1]
            error2: average of error in [time1+time2-avg_window, time1+time2]
            M__CE_mc1 : A dictionary with the results of comparing M__CE weight matrices
                       at time=0, and at time=time1 (see compare() above)
            M__CE_mc2 : A dictionary with the results of comparing M__CE weight matrices
                       at time=time1, and at time=time2
            AF__M_mc1 : A dictionary with the results of comparing AF__M weight matrices
                       at time=0, and at time=time1
            AF__M_mc2 : A dictionary with the results of comparing AF__M weight matrices
                       at time=time1, and at time=time2        
                     
    """
    cfg = args_dict['cfg']
    time1 = args_dict['time1']
    time2 = args_dict['time2']
    avg_window = args_dict['avg_window']
    err_thr = args_dict['err_thr']
    noisy_syns = args_dict['noisy_syns']
    decay = args_dict['decay']
    
    # First, seed the random generator
    np.random.seed() # trying to get seed from /dev/urandom
    
    # obtain a network with the given configuration
    (net, pops_dict, hand_coords, m_idxs, 
     t_pres, pds) = net_from_cfg(cfg, 
                                 M_mod=True,
                                 lowpass_SP=False,
                                 noisy_syns=noisy_syns,
                                 M__C_decay=False,
                                 AF__M_decay=False)
    
    # extract initial weight matrices
    M__CE_0 = [[syn.w for syn in net.syns[c] if 
               syn.preID in pops_dict['M']] for c in pops_dict['CE']]
    AF__M_0 = [[syn.w for syn in net.syns[m] if 
               syn.preID in pops_dict['AF']] for m in pops_dict['M']]

    # run the network with random targets for the first time
    start_time = time.time()
    times1, data1, plant_data1  = net.flat_run(time1)
    print('Random targets first run in %s seconds' % (time.time() - start_time))
    data1 = np.array(data1)
    
    # extract middle weight matrices
    M__CE_1 = [[syn.w for syn in net.syns[c] if 
               syn.preID in pops_dict['M']] for c in pops_dict['CE']]
    AF__M_1 = [[syn.w for syn in net.syns[m] if 
               syn.preID in pops_dict['AF']] for m in pops_dict['M']]
    
    # run the network with random targets for the second time
    start_time = time.time()
    times2, data2, plant_data2  = net.flat_run(time1)
    print('Random targets second run in %s seconds' % (time.time() - start_time))
    data2 = np.array(data2)
    
    # extract final weight matrices
    M__CE_2 = [[syn.w for syn in net.syns[c] if 
               syn.preID in pops_dict['M']] for c in pops_dict['CE']]
    AF__M_2 = [[syn.w for syn in net.syns[m] if 
               syn.preID in pops_dict['AF']] for m in pops_dict['M']]
    
    # joint the results of both simulations
    all_times = np.concatenate((times1, times2))
    all_data = np.concatenate((data1, data2), axis=1)
    all_plant_data = np.concatenate((plant_data1[0], plant_data2[0]), axis=0)
    
    # Calculate convergence time
    ## For presentation 'n', the target to be presented
    ## has coordinates hand_coords1[m_idxs[n]].
    coord_idxs = np.floor(all_times/t_pres).astype(int)
    des_coords = np.array(hand_coords)[m_idxs[coord_idxs],:]
    ## get distance vector
    l1 = net.plants[0].l_arm
    l2 = net.plants[0].l_farm
    theta_s = all_plant_data[:,0]
    theta_e = all_plant_data[:,2]
    phi = theta_s + theta_e # elbow angle wrt x axis
    # coordinates of hand and elbow
    c_hand = np.array([np.cos(theta_s)*l1 + np.cos(phi)*l2, 
                       np.sin(theta_s)*l1 + np.sin(phi)*l2]).transpose()
    error = np.linalg.norm(des_coords - c_hand, axis=1)
    
    ## get initial sum of avg_window elements in distance vector
    avg_win_idx = int(np.round(avg_window/net.min_delay))
    scaled_error = error / avg_win_idx
    win_sum = scaled_error[:avg_win_idx].sum()
    conv_t_idx = len(all_times)-1 # index of the convergence time
    ## calculate mean error at each time step
    for tidx in range(len(all_times)-avg_win_idx):
        if win_sum < err_thr:
            conv_t_idx = tidx
            break
        else:
            win_sum += scaled_error[tidx+avg_win_idx]
            win_sum -= scaled_error[tidx]
            
    t1_idx = int(np.round(time1/net.min_delay))
    error1 = scaled_error[t1_idx-avg_win_idx:t1_idx].sum()
    error2 = scaled_error[-avg_win_idx:].sum()
    # prepare return values
    rvals = {'conv_t' : all_times[conv_t_idx],
             'error1' : error1,
             'error2' : error2,
             'M__CE_mc1' : compare(np.array(M__CE_0), np.array(M__CE_1)),
             'M__CE_mc2' : compare(np.array(M__CE_1), np.array(M__CE_2)),
             'AF__M_mc1' : compare(np.array(AF__M_0), np.array(AF__M_1)),
             'AF__M_mc2' : compare(np.array(AF__M_1), np.array(AF__M_2))
    }     
    return rvals
        

In [7]:
# default configuration
def_cfg50 = {'b_e':4.00, 'M__C_lrate':20.26, 'sig1':0.49, 'SPF_w':1.50, 'M__C_w_sum':3.00, 'AL_thresh':0.30, 
           'g_e_factor':2.00, 'integ_amp':1.87, 'integ_decay':1.33, 'dely_diff':0.45, 'adapt_amp':5.00, 
           'C_tau_slow':49.90, 'SF_slope_factor':8.00, 'sig2':0.53, 'dely_low':0.77, 'n_evals':0.00 }
def_cfg26 = {'C_tau_slow':23.24, 'sig2':0.65, 'integ_decay':1.68, 'fitness':0.05, 'M__C_w_sum':3.03, 
             'n_evals':27.00, 'integ_amp':1.94, 'adapt_amp':5.00, 'dely_diff':0.61, 'SPF_w':1.50, 'M__C_lrate':24.82, 
             'sig1':0.07, 'g_e_factor':2.00, 'AL_thresh':0.38, 'dely_low':0.94, 'b_e':2.98, 'SF_slope_factor':8.00}
np.random.seed(123456)

In [8]:
# Load a saved population
import pickle
#fname = 'v3_nst_afx_pop_2020-08-09__20_39_gen26'
#fname = 'v3_nst_afx_pop_2020-09-04__09_52_cns1'
fname = 'v3_nst_afx_pop_2020-09-04__09_52_cns2'
#fname = 'v3_nst_afx_pop_2020-09-04__14_49_breaker'
with (open(fname, "rb")) as f:
    pop = pickle.load(f)
    f.close()

In [9]:
# select configuration
#cfg = def_cfg26
cfg = pop[0]

In [10]:
# changes to configuration
cfg['M__C_lrate'] = 10.

In [11]:
def par_test_net(cfg, time1=800, time2=1000,
                 avg_window= 200., err_thr=0.09, 
                 noisy_syns=False, decay=False,
                 n_tests=1, n_procs=1):
    """ Run test_net multiple times in parallel. 
    
        Args:
            cfg : parameter dictionary to initialize the network. 
            time1 : time for first simualtion period.
            time2 : time for second simualtion period.
            avg_window : window to obtain error averages. See below.
            err_thr : error threshold. See below.
            noisy_syns: if True, add synaptic noise
            decay: if noisy_syns, replace noise and normalization with decay
            n_tests : number of times to run test_net.
            n_procs : number of processes to use.
            
        Returns: A list of dictionaries with the following entries
            conv_t : convergence time. Smallest value of time such that the average
                     error in (conv_t, conv_t+avg_window) is less than err_thr.
            error1: average of error in [time1-avg_window, time1]
            error2: average of error in [time1+time2-avg_window, time1+time2]
            M__CE_mc1 : A dictionary with the results of comparing M__CE weight matrices
                       at time=0, and at time=time1 (see compare() above)
            M__CE_mc2 : A dictionary with the results of comparing M__CE weight matrices
                       at time=time1, and at time=time2
            AF__M_mc1 : A dictionary with the results of comparing AF__M weight matrices
                       at time=0, and at time=time1
            AF__M_mc2 : A dictionary with the results of comparing AF__M weight matrices
                       at time=time1, and at time=time2        
    """
    args = {'cfg' : cfg,
            'time1' : time1,
            'time2' : time2,
            'avg_window' : avg_window,
            'err_thr' : err_thr,
            'noisy_syns' : noisy_syns,
            'decay' : decay }
    args_list = [args] * n_tests
    with Pool(n_procs) as p:
        rlist = list(p.map(test_net, args_list))
        p.close()
        p.join()
    return rlist

In [12]:
# Run parallel tests
res_dict_list = par_test_net(cfg, # parameter dictionary
                             time1=1000.,    # time for first simualtion period.
                             time2 = 1200., # time for second simualtion period.
                             avg_window = 300., # window to obtain error averages. See below.
                             err_thr = 0.09, # error threshold. See below.
                             noisy_syns = True, # whether to add noise in synapses
                             decay = True,
                             n_tests = 4,   # number of times to run test_net.
                             n_procs = 4 )  # number of processes to use.

/home/z/projects/draculab/network.py:916: UserWarning: Integration method odeint substituted by Forward Euler in some units
  ' substituted by Forward Euler in some units', UserWarning)
/home/z/projects/draculab/network.py:916: UserWarning: Integration method odeint substituted by Forward Euler in some units
  ' substituted by Forward Euler in some units', UserWarning)
/home/z/projects/draculab/network.py:916: UserWarning: Integration method odeint substituted by Forward Euler in some units
  ' substituted by Forward Euler in some units', UserWarning)
/home/z/projects/draculab/network.py:916: UserWarning: Integration method odeint substituted by Forward Euler in some units
  ' substituted by Forward Euler in some units', UserWarning)


Random targets first run in 3807.474377155304 seconds
Random targets first run in 3842.5575540065765 seconds
Random targets first run in 3866.8189957141876 seconds
Random targets first run in 3893.953239917755 seconds
Random targets second run in 3861.804050207138 seconds
Random targets second run in 3846.688381433487 seconds
Random targets second run in 3859.92822432518 seconds
Random targets second run in 3871.281153678894 seconds


In [12]:
# Save the results dictionary
fname = "v3afxBt2_res"
fname += "_" + datetime.now().strftime('%Y-%m-%d__%H_%M')
with open(fname, 'wb') as f:
    pickle.dump([res_dict_list, cfg, time_rand, radial_rounds, avg_window, err_thr], f)
    f.close()

[{'R2': [0.8929816370847544,
   0.9688386965451752,
   0.8080942392009702,
   0.975610583570536,
   0.7151697323672797,
   0.9095645451172103,
   0.909711405355808,
   0.9427637253182222,
   0.6938445906638055,
   0.8930813804853641,
   0.7563900984537831,
   0.9064887526590691],
  'avg_ang_diff1': 1.1498903955892406,
  'avg_ang_diff2': 1.1699627797638745,
  'conv_t': 0.0,
  'last_error': 0.09793836635741589,
  'pref_vels1': array([[-0.89860509,  0.43003851],
         [-0.10785317,  0.98910368],
         [ 0.17103107, -0.98195084],
         [ 0.86625294, -0.4860126 ],
         [-0.50158632, -0.86035256],
         [ 0.61079902,  0.7838938 ],
         [-0.84745364,  0.52271015],
         [-0.07614548,  0.9926976 ],
         [ 0.20161816, -0.97678393],
         [ 0.72108717, -0.68713173],
         [-0.95279124, -0.29151878],
         [ 0.93055603,  0.35165178]]),
  'pref_vels2': array([[-0.0076922 , -0.00118882],
         [-0.00474701,  0.00022714],
         [-0.00107592, -0.00407278],
  

In [14]:
res_dict_list

[{'AF__M_mc1': {'angle': 1.6118287855786229,
   'match_prop': 0.4930555555555556,
   'nd_ratio': 1.7728505925811995},
  'AF__M_mc2': {'angle': 0.40395268331799955,
   'match_prop': 0.8611111111111112,
   'nd_ratio': 0.4028187251436396},
  'M__CE_mc1': {'angle': 1.441891264793858,
   'match_prop': 0.6527777777777778,
   'nd_ratio': 1.659730559228168},
  'M__CE_mc2': {'angle': 0.7403043189808289,
   'match_prop': 0.7916666666666666,
   'nd_ratio': 0.7263081911441701},
  'conv_t': 47.71500000000596,
  'error1': 0.11009530600561615,
  'error2': 0.09204513644809965},
 {'AF__M_mc1': {'angle': 1.5801491477616871,
   'match_prop': 0.5162037037037037,
   'nd_ratio': 1.7577734991318927},
  'AF__M_mc2': {'angle': 0.6728226570591211,
   'match_prop': 0.8287037037037037,
   'nd_ratio': 0.6605165178760292},
  'M__CE_mc1': {'angle': 1.4915577266767324,
   'match_prop': 0.4861111111111111,
   'nd_ratio': 1.655025480572592},
  'M__CE_mc2': {'angle': 0.8022542542586435,
   'match_prop': 0.79166666666666

In [13]:
# Present some numerical results

conv_ts = [dic['conv_t'] for dic in res_dict_list]
errors1 = [dic['error1'] for dic in res_dict_list]
errors2 = [dic['error2'] for dic in res_dict_list]
M__CE_match_props1 = [dic['M__CE_mc1']['match_prop'] for dic in res_dict_list]
M__CE_match_props2 = [dic['M__CE_mc2']['match_prop'] for dic in res_dict_list]
AF__M_match_props1 = [dic['AF__M_mc1']['match_prop'] for dic in res_dict_list]
AF__M_match_props2 = [dic['AF__M_mc2']['match_prop'] for dic in res_dict_list]
M__CE_nd_ratios1 = [dic['M__CE_mc1']['nd_ratio'] for dic in res_dict_list]
M__CE_nd_ratios2 = [dic['M__CE_mc2']['nd_ratio'] for dic in res_dict_list]
AF__M_nd_ratios1 = [dic['AF__M_mc1']['nd_ratio'] for dic in res_dict_list]
AF__M_nd_ratios2 = [dic['AF__M_mc2']['nd_ratio'] for dic in res_dict_list]
M__CE_angles1 = [dic['M__CE_mc1']['angle'] for dic in res_dict_list]
M__CE_angles2 = [dic['M__CE_mc2']['angle'] for dic in res_dict_list]
AF__M_angles1 = [dic['AF__M_mc1']['angle'] for dic in res_dict_list]
AF__M_angles2 = [dic['AF__M_mc2']['angle'] for dic in res_dict_list]


print("convergence times:")
print(conv_ts)
print("mean: %.2f, std: %.2f"%(np.mean(conv_ts), np.std(conv_ts)))
print("First stage:")
print("errors:")
print(errors1)
print("mean: %.2f, std: %.2f"%(np.mean(errors1), np.std(errors1)))
print("Second stage errors:")
print("errors:")
print(errors2)
print("mean: %.2f, std: %.2f"%(np.mean(errors2), np.std(errors2)))

print("\n Sign match proportions")
print("M__CE prop1 mean: %.2f, M__CE prop2 mean: %.2f"%
      (np.mean(M__CE_match_props1), np.mean(M__CE_match_props2)))
print("AF__M prop1 mean: %.2f, AF__M prop2 mean: %.2f"%
      (np.mean(AF__M_match_props1), np.mean(AF__M_match_props2)))

print("\n Ratio of matrix difference over mean of matrices norms")
print("M__CE nd_ratios1 mean: %.2f, M__CE nd_ratios2 mean: %.2f"%
      (np.mean(M__CE_nd_ratios1), np.mean(M__CE_nd_ratios2)))
print("AF__M nd_ratios1 mean: %.2f, AF__M nd_ratios2 mean: %.2f"%
      (np.mean(AF__M_nd_ratios1), np.mean(AF__M_nd_ratios2)))

print("\n Angle between matrices")
print("M__CE nd_angles1 mean: %.2f, M__CE nd_angles2 mean: %.2f"%
      (np.mean(M__CE_angles1), np.mean(M__CE_angles2)))
print("AF__M angles1 mean: %.2f, AF__M angles2 mean: %.2f"%
      (np.mean(AF__M_angles1), np.mean(AF__M_angles2)))

convergence times:
[920.4099999992354, 347.5949999997564, 1688.6250000136486, 691.1799999994439]
mean: 911.95, std: 492.58
First stage:
errors:
[0.09543113346069047, 0.16913137051314042, 0.1183421095303294, 0.0902429330064498]
mean: 0.12, std: 0.03
Second stage errors:
errors:
[0.11519942643607872, 0.10767519700785798, 0.07791018665136731, 0.09343509028211154]
mean: 0.10, std: 0.01

 Sign match proportions
M__CE prop1 mean: 0.53, M__CE prop2 mean: 0.74
AF__M prop1 mean: 0.51, AF__M prop2 mean: 0.91

 Ratio of matrix difference over mean of matrices norms
M__CE nd_ratios1 mean: 1.64, M__CE nd_ratios2 mean: 0.88
AF__M nd_ratios1 mean: 1.76, AF__M nd_ratios2 mean: 0.39

 Angle between matrices
M__CE nd_angles1 mean: 1.46, M__CE nd_angles2 mean: 0.92
AF__M angles1 mean: 1.58, AF__M angles2 mean: 0.39


Same history as `v3_nst_afxB`

---


In [15]:
# print used configuration
for dic in pop[0:1]:
    print('{',end='')
    for name in dic.keys():
        if name != 'fitness' or dic['fitness'] != None:
            print("\'%s\':%.2f, " % (name, dic[name]), end='')
    print('}\n')

pop[0]

{'fitness':0.06, 'integ_decay':1.68, 'SPF_w':2.25, 'dely_low':0.94, 'CI__CE_w':-2.00, 'sig2':0.65, 'CE__CI_w':0.95, 'n_evals':14.00, 'M__C_lrate':10.00, 'M__C_w_sum':1.75, 'integ_amp':1.94, 'g_e_factor':2.30, 'C__C_p_antag':0.55, 'C_tau_slow':23.24, 'SF_slope_factor':4.50, 'AL_thresh':0.06, 'C__C_antag':1.65, 'adapt_amp':5.00, 'b_e':3.50, 'sig1':0.07, 'dely_diff':0.61, }



{'AL_thresh': 0.05624690639116503,
 'CE__CI_w': 0.9450317641444606,
 'CI__CE_w': -2.0,
 'C__C_antag': 1.6505660923374748,
 'C__C_p_antag': 0.55,
 'C_tau_slow': 23.242490852056637,
 'M__C_lrate': 10.0,
 'M__C_w_sum': 1.75,
 'SF_slope_factor': 4.5,
 'SPF_w': 2.25,
 'adapt_amp': 5.0,
 'b_e': 3.5,
 'dely_diff': 0.606,
 'dely_low': 0.938,
 'fitness': 0.06045933847448534,
 'g_e_factor': 2.3,
 'integ_amp': 1.9404408352576688,
 'integ_decay': 1.676,
 'n_evals': 14,
 'sig1': 0.067,
 'sig2': 0.647}